In [1]:
import os

import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

from osgeo import gdal
from scipy.io import loadmat
from scipy.fft import ifft, fft, fftshift, ifftshift, fftfreq
from scipy.fft import ifft2, fft2
from scipy.constants import pi, c

In [2]:
from s1_level0_structs import (
    PRIMARY_HEADER,
    PRIMARY_HEADER_FIELDS,
    SECONDARY_HEADER,
    SECONDARY_HEADER_FIELDS,
    SUB_COMMUTATIVE_DATA_SERVICE,
    SUB_COMM_KEY_POS,
    SUB_COMM_KEY_VAL,
    SIMPLE_RECONSTRUCTION_METHOD,
    NORMALIZED_RECONSTRUCTION_LEVELS,
    THIDX_TO_SF_ARRAY,
    BRC_TO_THIDX,
    BRC_TO_M_CODE
)

from s1_level0_utils import (
    create_bit_string,
    read_and_pop,
    bit_len_map,
    huffman,
    huffman_3,
    huffman_4,
)

In [3]:
class Packet:
    def __init__(
        self,
        primary_header,
        secondary_header,
        user_data_field
    ):
        self.primary_header   = primary_header
        self.secondary_header = secondary_header
        self.raw_user_data    = user_data_field

        self.version_number        = int(primary_header['packet_version_number'], 2)
        self.type                  = int(primary_header['packet_type'], 2)
        self.secondary_header_flag = int(primary_header['secondary_header_flag'], 2)
        self.process_id            = int(primary_header['process_id'], 2)
        self.process_category      = int(primary_header['process_category'], 2)
        self.sequence_flags        = int(primary_header['sequence_flags'])
        self.sequence_count        = int(primary_header['packet_sequence_count'], 2)
        self.packet_data_length    = int(primary_header['packet_data_length'], 2)
        self.user_data_length      = (self.packet_data_length + 1) - 62

        self.num_quads   = int(self.secondary_header['num_quadratures'], 2)
        self.num_samples = 2 * self.num_quads

        self.test_mode      = int(self.secondary_header['test_mode'], 2)
        self.baq_mode       = int(self.secondary_header['baq_mode'], 2)
        self.block_length   = int(self.secondary_header['baq_block_length'], 2)
        self.num_baq_blocks = int(np.ceil(2 * self.num_quads / 256))

        self._set_data_format()
        self._decode_user_data_field()


    def _set_num_words_type_d(self):
        num_words_ie = int(np.ceil(self.block_length * self.num_quads / 16))
        num_words_qe = int(np.ceil((self.block_length * self.num_quads + 8 * self.num_baq_blocks) / 16))
        self.num_words = (num_words_ie, num_words_qe)


    def _get_type_d_signs_and_m_codes(
        self,
        bit_string: str,
        brc: int,
        last_block: bool
    ) -> ([int], [int], str):
        signs = []
        m_codes = []
        s_code_count = 0
        total_bits = 0
        num_s_codes = 128 if not last_block else self.num_quads - (128 * (self.num_baq_blocks - 1))
        while s_code_count < num_s_codes:
            sign, bit_string = read_and_pop(bit_string, 1)
            m_code, bit_string, bit_len = huffman(bit_string, brc)
            signs.append(int(sign, 2))
            m_codes.append(m_code)
            s_code_count += 1
            total_bits += (bit_len + 1)
        return  signs, m_codes, bit_string, total_bits
    

    def _decode_type_d_ie(self, bit_string):
        ie_bits = 0
        total_bits = 0
        for i in range(self.num_baq_blocks):
            brc, bit_string = read_and_pop(bit_string, 3)
            brc = int(brc, 2)
            if brc <= 4:
                last_block = i == self.num_baq_blocks - 1
                signs, m_codes, bit_string, total_bits = self._get_type_d_signs_and_m_codes(bit_string, brc, last_block)
                self.ie_signs.append(signs)
                self.ie_m_codes.append(m_codes)
                self.brc.append(brc)
            else:
                raise ValueError(f'Invalid BRC: {brc} at {i}')
            ie_bits += (total_bits + 3)
        bit_string, ie_offset = self._jump_to_next_word_boundary(bit_string, ie_bits)
        return bit_string, ie_bits + ie_offset


    def _decode_type_d_io(self, bit_string):
        io_bits = 0
        total_bits = 0
        for i in range(self.num_baq_blocks):
            brc = self.brc[i]
            last_block = i == self.num_baq_blocks - 1
            signs, m_codes, bit_string, total_bits = self._get_type_d_signs_and_m_codes(bit_string, brc, last_block)
            self.io_signs.append(signs)
            self.io_m_codes.append(m_codes)
            io_bits += total_bits
        bit_string, io_offset = self._jump_to_next_word_boundary(bit_string, io_bits)
        return bit_string, io_bits + io_offset


    def _decode_type_d_qe(self, bit_string):
        qe_bits = 0
        total_bits = 0
        for i in range(self.num_baq_blocks):
            brc = self.brc[i]
            threshold, bit_string = read_and_pop(bit_string, 8)
            last_block = i == self.num_baq_blocks - 1
            signs, m_codes, bit_string, total_bits = self._get_type_d_signs_and_m_codes(bit_string, brc, last_block)
            self.qe_signs.append(signs)
            self.qe_m_codes.append(m_codes)
            self.thresholds.append([threshold])
            qe_bits += (total_bits + 8)
        bit_string, qe_offset = self._jump_to_next_word_boundary(bit_string, qe_bits)
        return bit_string, qe_bits + qe_offset


    def _decode_type_d_qo(self, bit_string):
        qo_bits = 0
        total_bits = 0
        for i in range(self.num_baq_blocks):
            brc = self.brc[i]
            last_block = i == self.num_baq_blocks - 1
            signs, m_codes, bit_string, total_bits = self._get_type_d_signs_and_m_codes(bit_string, brc, last_block)
            self.qo_signs.append(signs)
            self.qo_m_codes.append(m_codes)
            qo_bits += total_bits
        bit_string, qo_offset = self._jump_to_next_word_boundary(bit_string, qo_bits)
        return bit_string, qo_bits + qo_offset


    def _jump_to_next_word_boundary(self, bit_string, num_bits):
        offset = num_bits % 16
        if offset == 0:
            return bit_string, offset
        next_word_boundary = 16 - offset
        filler, bit_string = read_and_pop(bit_string, next_word_boundary)
        return bit_string, next_word_boundary


    def _get_s_value(self, brc, threshold_index, m_code, sign):
        thidx_comp_flag = BRC_TO_THIDX[brc]
        m_code_comp_flag = BRC_TO_M_CODE[brc]
        if threshold_index <= thidx_comp_flag:
            if m_code < m_code_comp_flag:
                return (-1 ** sign) * m_code
            elif m_code == m_code_comp_flag:
                simple_recon = SIMPLE_RECONSTRUCTION_METHOD[1][brc][threshold_index]
                return ((-1) ** sign) * simple_recon
        elif threshold_index > thidx_comp_flag:
            normal_recon = NORMALIZED_RECONSTRUCTION_LEVELS[1][brc][m_code]
            sigma_factor = THIDX_TO_SF_ARRAY[threshold_index]
            return (-1 ** sign) * normal_recon * sigma_factor


    def _s_value_reconstruction(self):
        self.s_values = []
        for block_id in range(self.num_baq_blocks):
            brc = self.brc[block_id]
            threshold_index = int(self.thresholds[block_id][0], 2)
            block_len = len(self.ie_m_codes[block_id])
            for s_code_id in range(block_len):
                ie_s_value = self._get_s_value(brc, threshold_index, self.ie_m_codes[block_id][s_code_id], self.ie_signs[block_id][s_code_id])
                io_s_value = self._get_s_value(brc, threshold_index, self.io_m_codes[block_id][s_code_id], self.io_signs[block_id][s_code_id])
                qe_s_value = self._get_s_value(brc, threshold_index, self.qe_m_codes[block_id][s_code_id], self.qe_signs[block_id][s_code_id])
                qo_s_value = self._get_s_value(brc, threshold_index, self.qo_m_codes[block_id][s_code_id], self.qo_signs[block_id][s_code_id])
                component_s_values = [ie_s_value, io_s_value, qe_s_value, qo_s_value]
                self.s_values.append(component_s_values)


    def _align_s_values(self):
        complex_s_value = np.zeros((2 * self.num_quads, ), dtype=complex)
        for i in range(1, self.num_quads+1):
            ie, io, qe, qo = tuple(self.s_values[i-1])
            complex_s_value[2*i-2] = ie + 1j*qe
            complex_s_value[2*i-1] = io + 1j*qo
        self.complex_s_values = complex_s_value
    


    def _decode_type_d_data(self):    
        self._set_num_words_type_d()

        s_code_width = 0
        bit_string   = create_bit_string(self.raw_user_data)
        block_width  = (128 * self.block_length - 1) * self.num_baq_blocks

        self.brc        = []
        self.thresholds = []
        self.ie_signs, self.ie_m_codes = [], []
        self.io_signs, self.io_m_codes = [], []
        self.qe_signs, self.qe_m_codes = [], []
        self.qo_signs, self.qo_m_codes = [], []
    
        bit_string, self.num_ie_bits = self._decode_type_d_ie(bit_string)
        bit_string, self.num_io_bits = self._decode_type_d_io(bit_string)
        bit_string, self.num_qe_bits = self._decode_type_d_qe(bit_string)
        bit_string, self.num_qo_bits = self._decode_type_d_qo(bit_string)

        self.remaining_user_data_bits = len(bit_string)

        self._s_value_reconstruction()
        self._align_s_values()


    def _decode_user_data_field(self):
        if self.data_format == 'A':
            pass
        elif self.data_format == 'B':
            pass
        elif self.data_format == 'C':
            pass
        elif self.data_format == 'D':
            self._decode_type_d_data()


    def _set_data_format(self):
        self.test_mode = int(self.secondary_header['test_mode'], 2)
        self.baq_mode  = int(self.secondary_header['baq_mode'], 2)
        if self.baq_mode == 0:
            if self.test_mode % 3 == 0:
                self.data_format = 'A'
            else:
                self.data_format = 'B'
        elif self.baq_mode in [3, 4, 5]:
            self.data_format = 'C'
        elif self.baq_mode in [12, 13, 14]:
            self.data_format = 'D'
        else:
            raise ValueError(f'No Data Format with BAQ Mode: {self.baq_mode} and Test Mode: {self.test_mode}')

In [4]:
def get_header_dict(header_bytes, header_bit_lengths, header_field_names):
    read_and_pop = lambda bit_string, bit_length: (bit_string[0:bit_length], bit_string[bit_length:])
    bit_string = ''
    for byte in header_bytes:
        bit_string += '{:08b}'.format(byte)

    header_value_array = []
    for bit_len in header_bit_lengths:
        header_field_value, bit_string = read_and_pop(bit_string, bit_len)
        header_value_array.append(header_field_value) 

    index = 0
    header_dict = {}
    for field in header_field_names:
        if index >= len(header_field_names) or index >= len(header_value_array):
            raise ValueError('The number of field names and header values does not match.')
        header_dict[field] = header_value_array[index]
        index += 1

    return header_dict


def packet_generator(raw_data):
    secondary_header_length = 62
    while raw_data:
        primary_header_bytes = raw_data.read(6)
        primary_header = get_header_dict(primary_header_bytes, PRIMARY_HEADER, PRIMARY_HEADER_FIELDS)
        secondary_header_bytes = raw_data.read(62)
        secondary_header = get_header_dict(secondary_header_bytes, SECONDARY_HEADER, SECONDARY_HEADER_FIELDS)
        packet_data_length = primary_header['packet_data_length']
        user_data = None
        if packet_data_length:
            packet_data_length = int(packet_data_length, 2)
            user_data_length = (packet_data_length + 1) - secondary_header_length
            if user_data_length > 0:
                user_data = raw_data.read(user_data_length)
        yield Packet(
            primary_header = primary_header,
            secondary_header = secondary_header,
            user_data_field = user_data
        )

In [5]:
file_prefix  = 'sar_data/S1A_IW_RAW__0SDV_20240806T135224_20240806T135256_055093_06B68A_AE41.SAFE/'
data_file = 's1a-iw-raw-s-vv-20240806t135224-20240806t135256-055093-06b68a.dat'
raw_data_file = open(file_prefix + data_file, 'rb')

In [6]:
PacketGenerator = packet_generator(raw_data_file)

In [7]:
packet = next(PacketGenerator)

In [8]:
num_bytes = (packet.num_ie_bits + packet.num_io_bits + packet.num_qe_bits + packet.num_qo_bits) / 8
print(f"Total Octet Count: {num_bytes}")
user_data_bit_string = create_bit_string(packet.raw_user_data)
user_data_array_bytes = len(user_data_bit_string) / 8
print(f"User Data Length: {user_data_array_bytes}")
packet.secondary_header

Total Octet Count: 18558.0
User Data Length: 18560.0


{'coarse_time': '01010011110111001110110100101010',
 'fine_time': '0001111001001101',
 'sync_marker': '00110101001011101111100001010011',
 'data_take_id': '00001101011011010001010111000000',
 'ecc_number': '00001000',
 'na_1': '0',
 'test_mode': '000',
 'rx_channel_id': '0000',
 'instrument_configuration_id': '00000000000000000000000000000111',
 'sc_data_word_index': '00011011',
 'sc_data_word': '1011111011000011',
 'counter_service': '00000000000000111010101110010101',
 'pri_count': '00000000000000111011011010111000',
 'error_flag': '0',
 'na_2': '00',
 'baq_mode': '01100',
 'baq_block_length': '00011111',
 'na_3': '00000000',
 'range_decimation': '00001000',
 'rx_gain': '00001000',
 'tx_ramp_rate': '1000011001000101',
 'pulse_start_frequency': '0011000000101111',
 'pulse_length': '000000000000011110101111',
 'na_4': '000',
 'rank': '01001',
 'pri': '000000000101010101100011',
 'swst': '000000000000111001100001',
 'swl': '000000000011011010011011',
 'ssb_flag': '0',
 'polarisation': '

In [9]:
data_word_dicts = []
sub_comm_dict = SUB_COMM_KEY_VAL.copy()
initial_data_word_index = int(packet.secondary_header['sc_data_word_index'], 2)
key, pos = SUB_COMM_KEY_POS[initial_data_word_index]
pos = pos * 16
sub_comm_dict[key] = sub_comm_dict[key][0:pos] + packet.secondary_header['sc_data_word'] + sub_comm_dict[key][pos+16:]
num_packets = 150
for i in range(num_packets):
    if i % 10 == 0 and i > 0:
        print(f"Decoded {i} of {num_packets}.")
    packet = next(PacketGenerator)
    sc_data_word_index = int(packet.secondary_header['sc_data_word_index'], 2)
    if sc_data_word_index == initial_data_word_index:
        data_word_dicts.append(sub_comm_dict)
        sub_comm_dict = SUB_COMM_KEY_VAL.copy()
    key, pos = SUB_COMM_KEY_POS[sc_data_word_index]
    pos = pos * 16
    sub_comm_dict[key] = sub_comm_dict[key][0:pos] + packet.secondary_header['sc_data_word'] + sub_comm_dict[key][pos+16:]

Decoded 10 of 150.
Decoded 20 of 150.
Decoded 30 of 150.
Decoded 40 of 150.
Decoded 50 of 150.
Decoded 60 of 150.
Decoded 70 of 150.
Decoded 80 of 150.
Decoded 90 of 150.
Decoded 100 of 150.
Decoded 110 of 150.
Decoded 120 of 150.
Decoded 130 of 150.
Decoded 140 of 150.


In [10]:
data_word_dicts

[{'dummy_data': '0000000000000000',
  'x_axis_position': '1100000101000000011111011001001110000001000100110010010011001101',
  'y_axis_position': '1100000101010100011101111011000010110010011001110100111010000010',
  'z_axis_position': '0100000101001111000001110010101110111010001000111111000011011111',
  'x_axis_velocity': '11000101010001111010101110101001',
  'y_axis_velocity': '11000101010011101011011110011101',
  'z_axis_velocity': '11000101101111001101111001001110',
  'pod_data_stamp': '0000000001010011110111001110110100101001000000000000000000000000',
  'q0_quaternion': '10111110111010001101010001011110',
  'q1_quaternion': '10111111001111101110100100010011',
  'q2_quaternion': '10111110110000111111000100001001',
  'q3_quaternion': '00111110100110100001111110101100',
  'omega_x': '10111000100110000100011001100000',
  'omega_y': '10111010011101011110011110011111',
  'omega_z': '10111010000000100001100011110000',
  'data_time_stamp': '0000000001010011110111001110110100101000111000000

In [11]:
packet.complex_s_values

array([-1.907334-1.907334j, -1.907334-1.907334j, -5.722629-9.540432j, ...,
       -5.438955-1.81263j , -9.064215-5.438955j, -1.81263 -9.064215j])